This notebook is for developing the script for syncing data to a local machine, in order to speed up manual segmentation. 

In [ ]:
import pandas as pd

import os

import glob

from IPython.display import clear_output

In [ ]:
#load the relevant spreadsheet

matchedSubjects = pd.read_csv('./data/matched_diabetics_healthy_all.csv') #created by matching_diabetics_healthy_controls.ipynb

#convert to just a list of all of the feids
allFeids = pd.concat((matchedSubjects['diabetic feid'].dropna().astype('int'),matchedSubjects['healthy feid'].dropna().astype('int'))).reset_index(drop=True).apply(str)

In [ ]:

rawTopDir = './data/imaging_by_participant/'

def get_path_to_participant_raw(topDir,feid , suffix ,search = True):
    
    #search allows us to chase up files which aren't in the expected location... but is SLOW.
    
    tryIt = os.path.join(topDir, (feid[:2] + 'xxxxx'), feid, (feid + suffix) )
    
    if os.path.isfile(tryIt):        
        return tryIt
    elif search:
        print('searching...')
        allOptions = glob.glob('./data/**/'+ feid + suffix,recursive=True)
        clear_output()
        if len(allOptions) >= 1:
            return allOptions[0]
        else:
            return ''
    else:
        return ''
    
allRawFiles = allFeids.apply(lambda x: get_path_to_participant_raw(rawTopDir, str(x) , '_longaxis.zip' , search=False))

In [ ]:
allFeids[allRawFiles==''].count()


Having manually checked these, they are not on the GPU machine... Why not? :'(

anyway...

In [ ]:
allFeids[allRawFiles!='']

In [ ]:
bulkFile1 = '/workspace/storage/restricted-biobank/releases/REVISION_May2019/ID-27289/ukb27289.bulk'

bulkFile2 = '/workspace/storage/restricted-biobank/releases/REVISION_May2019/ID-29801/ukb29801.bulk'

In [ ]:
with open(bulkFile1,'r') as f:
    
    txt1 = f.read()
    
with open(bulkFile2,'r') as f:
    
    txt2 = f.read()
    

In [ ]:
scoutFound1 = allFeids.apply(lambda x: (x + ' 20207') in txt1)
scoutFound2 = allFeids.apply(lambda x: (x + ' 20207') in txt2)

laxFound1 = allFeids.apply(lambda x: (x + ' 20208') in txt1)
laxFound2 = allFeids.apply(lambda x: (x + ' 20208') in txt2)

In [ ]:
import numpy as np

In [ ]:
(allRawFiles!='').sum()

In [ ]:
(np.logical_and(scoutFound1,allRawFiles=='')).sum()

In [ ]:
(np.logical_and(scoutFound2,allRawFiles=='')).sum()

In [ ]:
(np.logical_and(laxFound2, allRawFiles=='')).sum()

In [ ]:
(np.logical_and(laxFound1, allRawFiles=='')).sum()

In [ ]:
listedinBulk1ButNoData = allFeids[np.logical_and(laxFound1, allRawFiles=='')]
listedinBulk2ButNoData = allFeids[np.logical_and(laxFound2, allRawFiles=='')]

In [ ]:
listedinBulk2ButNoData.astype(int).shape

In [ ]:
def checkDir(feid):
    
    checkDir = os.path.join(rawTopDir,feid[:2] + 'xxxxxx', feid)
    
    return os.path.isdir(checkDir)


listedinBulk2ButNoData.apply(checkDir)

In [ ]:
r9a1 = pd.read_csv('/images/imaging_by_participant/index_and_data_extract_unprocessed_r9a.csv')

r9a2 = pd.read_csv('/images/imaging_by_participant/inventory_heart_MRI_plus_values_r9a.csv')

In [ ]:
r9a2.set_index('feid').loc[allFeids[allRawFiles!=''],:]

In [ ]:
r9a1.set_index('f.eid').loc[allFeids[allRawFiles!=''],:]

In [ ]:
missing = allFeids[allRawFiles=='']

found = allFeids[allRawFiles!='']

In [ ]:
found.apply(lambda x: x in r9a1['f.eid'].values).all()

In [ ]:
missing.apply(lambda x: x in r9a1['f.eid'].values).all()

In [ ]:
missing.apply(lambda x: x in r9a1['f.eid'].values).any()

In [ ]:
missingButSomeInfo = missing[missing.apply(lambda x: x in r9a1['f.eid'].values)]

r9a1.set_index('f.eid').loc[missingButSomeInfo,:]

In [ ]:
found.apply(lambda x: x in r9a2['feid'].values).all()

In [ ]:
missing.apply(lambda x: x in r9a2['feid'].values).all()

In [ ]:
missing.apply(lambda x: x in r9a2['feid'].values).any()

In [ ]:
missingButSomeInfo = missing[missing.apply(lambda x: x in r9a2['feid'].values)]

r9a2.set_index('feid').loc[missingButSomeInfo,:].index

In [ ]:
from itertools import compress

In [ ]:

list(compress(allFeids.values,ins))

is there anything in particular about the dates on which people were imaged??

In [ ]:
original = pd.read_csv('./data/healthy_diabetics.csv',parse_dates = True)

In [ ]:
attendance = original.loc[:,['f.eid','bio.Date.of.attending.assessment.centre.0.imaging']].set_index(original['f.eid'].astype(str))

In [ ]:
import datetime

attendance = attendance['bio.Date.of.attending.assessment.centre.0.imaging'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y'))

In [ ]:
original.set_index(original['f.eid'].astype('str')).loc[missing,'bio.Date.of.attending.assessment.centre.0.imaging.y']#.apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y'))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(attendance[missing],alpha = 0.5)

plt.hist(attendance[found],alpha = 0.5)

In [ ]:
[c for c in original.columns if 'cent' in c]